In [1]:
import tensorflow
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
from clean_text import clean_text

import string
import re
import preprocessor as p
from spacy.lang.en import stop_words as spacy_stopwords  # we use spacy's list of stop words to clean our data
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import SVC

2022-01-07 16:18:36.318442: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-07 16:18:36.318484: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Text representation byWord Embeddings Transformers -> Universal Sentence Encoder

In [3]:
# load pretrained model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [4]:
from sklearn.datasets import fetch_20newsgroups
# Prepare data

corpus = fetch_20newsgroups()
categories = fetch_20newsgroups()["target_names"]
train_set = fetch_20newsgroups(subset = 'train', categories = categories)
test_set = fetch_20newsgroups(subset = 'test', categories = categories)

X_train = train_set.data
y_train = train_set.target

X_test = test_set.data
y_test = test_set.target


sample = 35

# show sample of train_set
print("LABEL: {}".format(categories[y_train[sample]]))
print(X_train[sample])


LABEL: rec.sport.hockey
From: dchhabra@stpl.ists.ca (Deepak Chhabra)
Subject: Re: Goalie masks
Nntp-Posting-Host: stpl.ists.ca
Organization: Solar Terresterial Physics Laboratory, ISTS
Lines: 21

In article <120666@netnews.upenn.edu> kkeller@mail.sas.upenn.edu (Keith Keller) writes:
>My vote goes to John Vanbiesbrouck.  His mask has a skyline of New York
>City, and on the sides there are a bunch of bees (Beezer).  It looks
>really sharp.

Funny you should mention this; one time on HNIC Don Cherry pointed out
Vanbiesbrouck's mask.  He _hated_ it.  I think he said something to the effect
of:
"You see?  He was great last year; now he goes out and gets that dopey mask 
and he can't stop a beachball!"

You may or may not take Cherry seriously at all, but I cracked up when I heard
it.

I think Ed Belfour has the current best mask in the NHL btw.  I also like
Moog's, and I'll give Fuhr's new one an honourable mention, although I haven't
seen it closely yet (it looked good from a distance!).  

In [5]:
# Show the text cleaner work
cleaned = clean_text(X_train[sample])

print("LABEL: {}".format(categories[y_train[sample]]))
print(cleaned)

LABEL: rec.sport.hockey
from dchhabra stpl ists ca deepak chhabra subject re goalie masks nntp posting host stpl ists ca organization solar terresterial physics laboratory ists lines number in article number netnews upenn edu kkeller mail sas upenn edu keith keller writes my vote goes to john vanbiesbrouck his mask has a skyline of new york city and on the sides there are a bunch of bees beezer it looks really sharp funny you should mention this one time on hnic don cherry pointed out vanbiesbrouck s mask he _hated_ it i think he said something to the effect of you see he was great last year now he goes out and gets that dopey mask and he can t stop a beachball you may or may not take cherry seriously at all but i cracked up when i heard it i think ed belfour has the current best mask in the nhl btw i also like moog s and i ll give fuhr s new one an honourable mention although i haven t seen it closely yet it looked good from a distance what s also neat is chevaldae s in detroit they c

https://www.nbshare.io/notebook/197284676/Word-Embeddings-Transformers-In-SVM-Classifier-Using-Python/ -> clue

In [6]:
# Clean X_train and X_test
X_train = [clean_text(x_train) for x_train in X_train]
X_test = [clean_text(x_test) for x_test in X_test]


In [7]:
# we just feed in the list of sentences, and we get the vector representation of each sentence
# we don't have enough memory to apply embeddings in one shot,
# so we have to split the data into batches and concatenate them later

def transform_data(data, split):
    splits = np.array_split(data, split)
    l = list()
    for split in splits:
        l.append(embed(split))
    data = tensorflow.concat(l, axis=0)
    del l

    return data

# Do not have enough memory
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print(f'Data instances before transform {len(X_test)}')

X_test = transform_data(X_test, 1)
print(f'Data instances after transform {len(X_test)}')

    

Data instances before transform 7532


# MI SIE CRASHUJE SSYTEM NA CPU, NAWET JAK PROBUJE PRZEKSZTAŁCIĆ JEDNĄ DANĄ NA RAZ XD, spróbuj to puścić. Jak się uda to trzeba tu niżej te wszystkie klasyfikatory jak w pliku `classify_text_first.ipynb` przelecieć i sprawdzić, poniżej wrzucam kod jak to mnie wiecej bedzie wyglądać i porobić wykresy i macierze pomyłek

In [ ]:
from sklearn.metrics import classification_report
clf = SVC() # z parametrami
clf.fit(X_train, y_train)
y_pred = clf.predict(np.array(X_test))

# I wykresy porobić oraz metryki jak w poprzednim pliku 